# From Experiments to Data
## Computational Methods in Psychology (and Neuroscience)
### Psychology 4500/7559 --- Fall 2020
By: Per B. Sederberg, PhD



# Lesson Objectives

Upon completion of this lesson, students should have learned:

1. How to polish up a complete experiment
2. How SMILE stores data
3. How to read in slog files
4. An introduction to Pandas


# Review list generation solution

- Let's go through the list gen code you'll use for your next assignment!

# Updating SMILE

- First you can test whether there is a new version Kivy, which is the primary dependency of SMILE:

```bash
conda install -c conda-forge kivy
```

- Then you can update SMILE right from the GitHub repository (note the upgrade option at the end):

```bash
pip install git+https://github.com/compmem/smile --upgrade
```

# Subject Info

- When collecting data from a lot of participants in a lab setting, you need an easy way to enter subject information once the experiment has started.

In [ ]:
from smile.common import *
from smile.startup import InputSubject

exp = Experiment(show_splash=False, resolution=(1024,768))

InputSubject('Flanker')

Label(text='Hello!')
with UntilDone():
    KeyPress()
    
exp.run()

# Instructions

- Participants need some guidance as to how they should perform the task
- This can be just text via `Label` states, but even better would be a visual description, and even better would be a full tutorial.
- ***NOTE: If you provide a tutorial, do not use items that you will use for the actual task!***

In [ ]:
from smile.common import *

exp = Experiment(show_splash=False, resolution=(1024,768))

# Labels can be multi-line with markup!
with Parallel():
    Label(text="This is multi-line text aligned to the left", 
          font_size=50, text_size=(400, None),
          left=exp.screen.left)
    Label(text="This is multi-line text aligned to the right", 
          font_size=50, text_size=(400, None), 
          right=exp.screen.right, halign='right')
with UntilDone():
    KeyPress()
    
exp.run()

# Scaling to different screen sizes/densities

- Screen sizes and pixel densities can vary across devices (especially on laptops, tablets, and phones)
- But all sizes in Kivy/SMILE are defined by number of pixels
- We provide a means to scale across devices:

In [ ]:
# load in smile states
from smile.common import *
from smile.scale import scale as s

# create an experiment instance (pick different resolutions)
resolution = (800,600)
#resolution = (1024,768)
exp = Experiment(show_splash=False, resolution=resolution,
                 scale_up=True, scale_down=True, scale_box=(400, 300))
Wait(.5)
with Parallel():
    # add a scaled rectangle
    Rectangle(width=s(250),height=s(250))
    
    # and a fixed size rectangle
    Rectangle(width=100, height=100, color='blue')
with UntilDone():
    KeyPress()

exp.run()

# Logging and Slogging

- SMILE states automatically log themselves, so you rarely will lose information if you forget to log it.
- It's still much easier to analyze a well-organized log file from your experiment.
- Make use of the `Log` state to save out data.
- All logs are stored in `.slog` files, which is short for SMILE Log.
- These are read in as a list of dictionaries (or a dict-list):

```python
from smile.log import log2dl
dl = log2dl('flanker_log.slog')
```

# Where have all the data gone?

- All the data files are stored in the `data` directory
  - NOTE: Even when you don't specify a subj, it saves to a `test000` directory, so you can delete that sometimes to save space.
- There is a hierarchical structure to the data directory:
  - Experiment -> SubjID -> Date/Time

In [ ]:
!ls data

# Let's continue learning by building together!

- Remaining features to add:
  - Instructions
  - Loop over blocks and then trials
  - Verify logging

# List Gen Function

- NOTE: New addition for multiple blocks of trials

In [ ]:
import random 
import copy

# define the conditions
conds = [{'condition': 'congruent',
          'direction': 'left',
          'stimulus': '<<<<<<<'
         },
         {'condition': 'congruent',
          'direction': 'right',
          'stimulus': '>>>>>>>'
         },
         {'condition': 'incongruent',
          'direction': 'left',
          'stimulus': '>>><>>>'
         },
         {'condition': 'incongruent',
          'direction': 'right',
          'stimulus': '<<<><<<'
         },
         {'condition': 'neutral',
          'direction': 'left',
          'stimulus': '===<==='
         },
         {'condition': 'neutral',
          'direction': 'right',
          'stimulus': '===>==='
         },]

# specify number of reps of these conditions
num_reps = 2
num_blocks = 3

# loop and create the blocks
blocks = []
for b in range(num_blocks):
    # loop and create the list
    trials = []
    for i in range(num_reps):
        # extend the trials with copies of the conditions
        trials.extend(copy.deepcopy(conds))
        
    # shuffle the trials
    random.shuffle(trials)
    
    # append the trials to the blocks
    blocks.append(trials)

print(blocks)

# Goal for task

## High level structure

- Present instructions
- Loop over blocks
- Within each block, loop over trials

## Trial structure

- Present the correct stimulus as text on the screen
- Wait for a response
- Remove the stimulus
- Wait for an inter-stimulus interval
- Log the result of the trial

In [ ]:
from smile.common import *
from smile.scale import scale as s

# config section
font_size = s(75)
resp_keys = ['F', 'J']
resp_map = {'left': 'F', 'right': 'J'}
ISI_dur = 0.5
ISI_jitter = 0.5
LOC_X_jitter = s(200)
LOC_Y_jitter = s(100)
inst_font_size = s(25)
inst_text = """[size=40]FLANKER INSTRUCTIONS[/size]
    
Press ENTER key to continue."""

# create the experiment
exp = Experiment(name='FLANKER', show_splash=False, 
                 fullscreen=False,
                 resolution=(1024, 768))

@Subroutine
def Instruct(self):
    # show the instructions
    Label(text=inst_text, font_size=inst_font_size,
          text_size=(exp.screen.width*0.75, None),
          markup=True)
    with UntilDone():
        KeyPress(keys=['ENTER'])

@Subroutine
def Trial(self, block_num, trial_num, cur_trial):
    # pick the new stimulus location
    self.location = (jitter(self.exp.screen.center_x-LOC_X_jitter,
                            LOC_X_jitter*2),
                     jitter(self.exp.screen.center_y-LOC_Y_jitter,
                            LOC_Y_jitter*2))
    # present the stimulus
    stim = Label(text=cur_trial['stimulus'],
                 font_size=font_size,
                 center=self.location)
    with UntilDone():
        # make sure the stimulus has appeared on the screen
        Wait(until=stim.appear_time)
        
        # collect a response (with no timeout)
        kp = KeyPress(keys=resp_keys, 
                      base_time=stim.appear_time['time'],
                      correct_resp=Ref.object(resp_map)[cur_trial['direction']])
    
    # wait the ISI with jitter
    Wait(ISI_dur, jitter=ISI_jitter)

    # log the result of the trial
    Log(name='flanker', 
        log_dict=cur_trial,
        block_num=block_num,
        trial_num=trial_num,
        stim_on=stim.appear_time,
        resp=kp.pressed,
        resp_time=kp.press_time,
        rt=kp.rt,
        correct=kp.correct,
        location=self.location
       )
    
# show the instructions
Instruct()
Wait(0.5)

# loop over the blocks
with Loop(blocks) as block:
    # make sure they are ready to continue
    Label(text='Press the ENTER key to\nstart the next block.', 
          font_size=font_size, halign='center')
    with UntilDone():
        KeyPress(keys=['ENTER'])

    # add in some delay before the start of the block
    Wait(ISI_dur, jitter=ISI_jitter)
    
    # loop over the trials
    with Loop(block.current) as trial:
        Trial(block.i, trial.i, trial.current)

# run the experiment
exp.run()

# Read in some data from the logs

In [ ]:
from smile.log import log2dl
log2dl('data/SMILE/test000/20201015_002203/log_flanker_0.slog')

## Assignment before next class

- Your SMILE experiment is due next Thursday!

### See you next week!!!